In [1]:
from parse_data import get_data
from generator import Generator
from discriminator import Discriminator
import torch

df = get_data()

def make_data(df):
    
    x_train, y_train = [], []
    prev = -1
    count = 1
    m = df.max()[0]
    maxtime = df.max()[1]
    print("Max value: ", m)
    
    for row in df.values:
        x_train.append([prev/m, count/maxtime])
        y_train.append([row[0]/m])
        prev = row[0]
        count += 1
    return torch.tensor(x_train, dtype=torch.double),torch.tensor(y_train, dtype=torch.double)

make_data(df)

Max value:  1000


/tmp/ipykernel_58662/182989242.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]
/tmp/ipykernel_58662/182989242.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  maxtime = df.max()[1]


(tensor([[-1.0000e-03,  7.7509e-08],
         [ 1.0000e+00,  1.5502e-07],
         [ 4.6000e-01,  2.3253e-07],
         ...,
         [ 1.0000e+00,  6.3170e-04],
         [ 4.6000e-01,  6.3177e-04],
         [ 5.9000e-01,  6.3185e-04]], dtype=torch.float64),
 tensor([[1.0000],
         [0.4600],
         [1.0000],
         ...,
         [0.4600],
         [0.5900],
         [0.5900]], dtype=torch.float64))

### *Make this cell "code" to enable warnings*
### *Only used for debugging*
import warnings
warnings.filterwarnings('error')

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data
from IPython.display import clear_output
import torch.nn as nn


generator = Generator()
discriminator = Discriminator()

optimizer_g = optim.Adam(generator.parameters(), lr=0.01)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.01)

loss = nn.MSELoss()

batch_size = 50
x_d, y_d = make_data(get_data())
loader = data.DataLoader(data.TensorDataset(x_d,y_d), batch_size=batch_size)
epochs = 1000


true = torch.tensor([1.0]*batch_size).unsqueeze(1).double()
false = torch.tensor([0.0]*batch_size).unsqueeze(1).double()
for e in range(epochs):

    #clear_output()
    # train generator
    clear_output()

    for _ in range(5):

        generator.lstm_zero()
        
        res = []
        for x, _ in loader:
            if x.size()[0] < batch_size:
                continue
            generator.init_state()
            discriminator.random_state()
            y_pred = generator(x)
            d = discriminator(y_pred)
            res.append(loss(d, true))

        l = res[0]
        for i in res[1:]:
            l += i
        print("Generator loss: ", l.item())
        optimizer_g.zero_grad()
        l.backward()
        optimizer_g.step()


    # Train discriminator
    for _ in range(5):
    
        optimizer_d.zero_grad()
        generator.lstm_zero()
        res = []

        for x, y in loader:
            if x.size()[0] < batch_size:
                continue
            generator.init_state()
            discriminator.random_state()
            y_pred = generator(x).detach()
            res.append(loss(discriminator(y_pred), false))
            discriminator.random_state()
            res.append(loss(discriminator(y), true))

        l = res[0]
        for i in res[1:]:
            l += i
        print("Discriminator loss: ", l.item())
        optimizer_d.zero_grad()
        l.backward()
        optimizer_d.step()
    
    
        


Generator loss:  41.32178910252702
Generator loss:  41.14333931086184
Generator loss:  41.00515172547595
